<a href="https://colab.research.google.com/github/rencdr/python.RL.CartPole/blob/master/RL_CartPole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gym

import gym
import numpy as np

In [ ]:
env = gym.make("CartPole-v1")

In [4]:
# Q-değer tablosunu başlat
# Gözlem alanını discretize etmek için örnek: [pozisyon, hız, açı, açı hızı]
# Bu örnekte, her bir özellik için 10 aralık seçiyoruz
state_space_bins = [10, 10, 10, 10]
action_space_bins = [env.action_space.n]
Q_table = np.random.uniform(low=-1, high=1, size=(state_space_bins + action_space_bins))

In [5]:
# Q-learning parametreleri
learning_rate = 0.1
discount_factor = 0.99
exploration_prob = 0.2

In [6]:
# Eğitim parametreleri
num_episodes = 1000

In [ ]:
# Eğitim Döngüsü
for episode in range(num_episodes):
    state = env.reset()
    state_bins = [np.digitize(state[i], bins=np.linspace(-4.8, 4.8, state_space_bins[i]-1)) for i in range(len(state))]
    total_reward = 0

    while True:
        if np.random.rand() < exploration_prob:
            action = env.action_space.sample()
        else:
            action = int(np.argmax(Q_table[state_bins]))

        action = np.clip(action, 0, env.action_space.n - 1)

        next_state, reward, done, _ = env.step(action)
        next_state_bins = [np.digitize(next_state[i], bins=np.linspace(-4.8, 4.8, state_space_bins[i]-1)) for i in range(len(next_state))]
        Q_table[state_bins + [action]] += learning_rate * (reward + discount_factor * np.max(Q_table[next_state_bins]) - Q_table[state_bins + [action]])

        state = next_state
        state_bins = next_state_bins
        total_reward += reward

        if done:
            break

    print(f"Episode: {episode + 1}, Total Reward: {total_reward}")




In [ ]:
# Eğitim sonrasında ajanın performansını gözlemleme
state = env.reset()
state_bins = [np.digitize(state[i], bins=np.linspace(-4.8, 4.8, state_space_bins[i]-1)) for i in range(len(state))]
total_reward = 0

while True:
    # Düzeltilmiş kısım
    action = int(np.argmax(Q_table[state_bins]))  # Q-değerlerinden en büyük olan indeksi al
    action = np.clip(action, 0, env.action_space.n - 1)  # Aksiyon uzayının sınırları dışına çıkmasını engelle
    next_state, reward, done, _ = env.step(action)
    next_state_bins = [np.digitize(next_state[i], bins=np.linspace(-4.8, 4.8, state_space_bins[i]-1)) for i in range(len(next_state))]

    state = next_state
    state_bins = next_state_bins
    total_reward += reward

    env.render()

    if done:
        break

print(f"Eğitilmiş Ajanın Toplam Ödülü: {total_reward}")